In [ ]:
# **ResNet18 with CIFAR-10 — 학습/추론/시각화**

## 학습 목표
- Residual Connection(잔차 연결)의 구조와 원리를 이해하고, ResNet-18 모델을 CIFAR-10 이미지 분류 작업에 맞게 구현 및 최적화할 수 있도록 실습합니다.
- 데이터 전처리부터 모델 학습, 성능 평가, 그리고 Class Activation Map(CAM) 등을 활용한 추론 결과 시각화까지의 ResNet 파이프라인 전 과정을 익힙니다.

---

### 구성 개요
1. **CIFAR-10 로드** + 입력 이미지 크기 확인
2. **ResNet18(CIFAR-friendly)** 구성
3. **학습(Training)** + 체크포인트 저장/재개(있으면 +10 epoch, 없으면 20 epoch)
4. **추론(Inference)** + 예측/신뢰도 확인
5. **Confusion Matrix / 클래스별 정확도 / 오분류 샘플** 분석
6. **Activation Map** 시각화 (layer1 vs layer4 비교)
7. **Conv Kernel** 시각화 (초기/중간 레이어)
8. **Grad-CAM** 시각화 (Top-K 정분류 + Top-K 오분류: 원본과 Overlay 함께)
9. (추가 실습) **Receptive Field** 근사 계산 + 의미 설명
10. (추가 실습) **Embedding(t-SNE)** 시각화
11. (추가 실습) **ONNX Export + 모델 구조 출력** (Netron 비교용)

> 권장 실행: 위에서 아래로 순차 실행


In [ ]:
import importlib.util
import subprocess
import sys

def ensure_package(pkg_name: str, import_name=None):
    """
    패키지 설치 여부를 확인하고, 설치되어 있지 않으면 설치합니다.
    """
    name = import_name or pkg_name
    # 패키지가 설치되어 있는지 확인
    if importlib.util.find_spec(name) is None:
        print(f"[install] {pkg_name} 라이브러리를 설치 중입니다... (import name: {name})")
        try:
            # -q 옵션을 추가하여 설치 과정을 간결하게 유지할 수 있습니다.
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg_name])
            print(f"[success] {pkg_name} 설치 완료.")
        except subprocess.CalledProcessError as e:
            print(f"[error] {pkg_name} 설치 실패: {e}")
    else:
        print(f"[ok] {pkg_name} 이미 설치되어 있습니다.")

# 설치가 필요한 패키지 리스트 (패키지명, 임포트명)
# 임포트명이 패키지명과 다른 경우 튜플로 지정합니다.
packages = [
    ("torch", "torch"),
    ("torchvision", "torchvision"),
    ("matplotlib", "matplotlib"),
    ("onnx", "onnx"),
]

# 루프를 돌며 확인 및 설치
for pkg, imp in packages:
    ensure_package(pkg, imp)

In [ ]:
import os  # path/환경 확인
import random  # seed 고정
import time  # epoch 시간 측정
import numpy as np  # 수치 계산
import torch  # PyTorch 본체
import torch.nn as nn  # 모델 레이어
import torch.nn.functional as F  # softmax 등 함수
from torch.utils.data import DataLoader  # 데이터 로더
import torchvision  # vision dataset/util
import torchvision.transforms as T  # transform
import torchvision.models as models  # resnet18
import matplotlib.pyplot as plt  # 시각화

print('torch:', torch.__version__)  # torch 버전 출력
print('torchvision:', torchvision.__version__)  # torchvision 버전 출력
print('cuda available:', torch.cuda.is_available())  # GPU 사용 가능 여부
if torch.cuda.is_available():
    print('gpu:', torch.cuda.get_device_name(0))  # GPU 이름 출력

try:
    get_ipython().system('nvidia-smi -L')  # GPU 목록 확인(가능한 경우)
except Exception as e:
    print('nvidia-smi not available:', e)  # nvidia-smi가 없는 환경이면 무시

def seed_everything(seed: int = 42):
    random.seed(seed)  # python random seed
    np.random.seed(seed)  # numpy seed
    torch.manual_seed(seed)  # torch CPU seed
    torch.cuda.manual_seed_all(seed)  # torch GPU seed
    torch.backends.cudnn.deterministic = False  # 성능 우선(완전 재현성 X)
    torch.backends.cudnn.benchmark = True  # 입력 크기 고정이면 속도 향상

seed_everything(42)  # seed 고정(실험 재현성)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 실행 디바이스 선택
device  # 디바이스 출력


In [ ]:
---
## 1) Dataset: CIFAR-10 로드 + 입력 크기 확인
- **목적:** 학습/평가에 사용할 데이터셋을 준비합니다.
- **관찰 포인트**
    - CNN 입력은 **(C,H,W)** 텐서이며, CIFAR-10은 **32×32** 입니다.
    - augmentation(RandomCrop/Flip)을 적용합니다.
    - CIFAR-10 데이터셋 구성
       - Number of Classes: 10 (e.g., airplane, dog, cat, frog, etc.)
       - Image Distribution: The dataset is class-balanced, with 6,000 images per class (5,000 for training and 1,000 for testing).
       - Image Format: All images are 32x32 pixel color images (3 RGB channels).

In [ ]:
# CIFAR-10에서 널리 쓰는 normalize 값
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)  # 채널별 평균(R,G,B)
CIFAR10_STD  = (0.2023, 0.1994, 0.2010)  # 채널별 표준편차(R,G,B)

train_tfms = T.Compose([  # train transform(augmentation 포함)
    T.RandomCrop(32, padding=4),  # 32x32를 padding 후 random crop
    T.RandomHorizontalFlip(),  # 좌우 반전
    T.ToTensor(),  # PIL -> torch tensor (C,H,W), 0~1
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD),  # 정규화, -2.5~2.5
])

test_tfms = T.Compose([  # test transform(augmentation 없음)
    T.ToTensor(),  # 텐서 변환
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD),  # 정규화
])

data_root = './data'  # 데이터 저장 경로
train_set = torchvision.datasets.CIFAR10(root=data_root, train=True, downloads=True, transform=train_tfms)
test_set = torchvision.datasets.CIFAR10(root=data_root, train=False, downloads=True, transform=test_tfms)

class_names = train_set.classes  # 클래스 이름 목록
num_classes = len(class_names)  # 클래스 개수(10)
print('classes:', class_names)  # 클래스 출력

# 이미지 사이즈 출력
x0, y0 = train_set[0]  # 한 샘플 로드(이미 transform 적용된 텐서)
print('Sample image tensor shape (C,H,W):', tuple(x0.shape))  # (3,32,32)

batch_size = 256  # 배치 크기
num_workers = min(8, os.cpu_count() or 2)  # dataloader worker 수(환경에 맞게)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)  # test loader

print('Total batch size (train: %d, test: %d)' % (len(train_loader), len(test_loader)))  # 배치 개수 확인


In [ ]:
def denorm(x):
    mean = torch.tensor(CIFAR10_MEAN).view(1,3,1,1)
    std = torch.tensor(CIFAR10_STD).view(1,3,1,1)
    return x * std + mean

images, label = next(iter(train_loader))
images_dn = denrom(images[:16]).clamp(0,1)

grid = torchvision.utils.make_grid(images_dn, nrow=8)  # 그리드로 묶기(C,H,W)
plt.figure(figsize=(12,4))
plt.imshow(grid.permute(1,2,0))  # CHW -> HWC 로 변환 후 표시
plt.axis('off')
plt.title('CIFAR-10 samples (denormalized)')
plt.show()

print('labels:', [class_names[int(x)] for x in labels[:16]])  # 라벨 이름 출력

In [ ]:
def build_resnet18_for_cifar10(num_classes: int =10):
    model = models.resnet18(weight=None)
    print("\n" + "="*40)
    print("=== PyTorch model structure (Before) ===")  # 모델 구조 출력
    print("="*40 + "\n")
    print(model)  # 구조 출력
    
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
model = build_resnet18_for_cifar10(num_classes).to(device)
    
print("\n" + "="*40)
print("=== PyTorch model structure (After) ===")  # 모델 구조 출력
print("="*40 + "\n")
print(model)  # 구조 출력

num_params = sum(p.numel()) for p in model.parameters())
print(f'params: {num_params:,}')  # 출력

In [ ]:
---
## 3) 학습 유틸리티
- **목적:** 학습/평가 루프를 재사용 가능하게 만들어 실험을 단순화합니다.
- **관찰 포인트**
  - `model.train()` vs `model.eval()` 차이
  - AMP(Automatic Mixed Precision) 사용
     - 계산이 복잡한 곳은 FP32를 쓰고, 단순한 곳은 FP16을 섞어서 사용
     - GradScaler의 역할: FP16을 쓰면 기울기(Gradient) 값이 너무 작아져서 사라질 위험이 있음 (언더플로우)
        - Scaling: 손실(Loss) 값에 아주 큰 값을 곱해 기울기를 크게 키움
        - Backprop: 커진 상태로 역전파를 수행하여 값이 사라지지 않게 보호
        - Unscaling: 업데이트 직전에 다시 원래 비율로 줄여서 가중치에 반영

In [ ]:
from dataclasses import dataclass  # 설정을 깔끔하게 묶기 위한 dataclass

@dataclass
class TrainConfig:
    base_epochs_if_new: int = 20  # 처음 실험 시 epoch 수
    extra_epochs_if_resume: int = 10  # 체크포인트 존재 시 추가 실험 epoch 수
    lr: float = 0.1  # learning rate
    weight_decay: float = 5e-4  # L2 정규화
    momentum: float = 0.9  # SGD momentum
    label_smoothing: float = 0.0  # 필요 시 label smoothing, 예) [0, 1, 0] ->  [0.05, 0.9, 0.05]

cfg = TrainConfig()

def accuracy_top1(logits, targets):
    preds = logits.argmax(dim=1) # 가장 큰 logit의 클래스 선택, logits = [Batch, Classes], preds = [Batch]
    return (preds==targets).float().mean().item()

scaler = torch.amp.GradScaler(enable=torch.cuda.is_available())
print (scaler)

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_acc = 0.0, 0.0
    n=0
    t0= time.time()
    
    for images, labels in loader:
        images = imgaes.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        with torch.amp.autocast(device_type=device.type, enabled=torch.cuda.is_available()):  # AMP autocast
            logits = model(images)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
            
        """
        일반 학습
        optimizer.zero_grad()
         ...
        loss.backward() 
        optimizer.step() 
       
        AMP학습 (Mixed Precision)
        optimizer.zero_grad()
        ...
        scaler.scale(loss).backward() # gradient를 스케일링 
        scaler.step(optimizer) # 스케일링 해제 후 optimizer.step() 호출 
        scaler.update() # 다음 step을 위한 scale 값 업데이트 
        """
        
        bs = images.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy_top1(logits.detach(), labels) * bs
        n += bs
    dt = time.time() - t0  # epoch 수행 시간
    return total_loss / n, total_acc / n, dt  # 평균 loss/acc/시간
        
@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_acc = 0.0, 0.0  # 누적
    n = 0  # 샘플 수
    for images, labels in loader :
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        logits = model(images)
        loss = criterion(logits, labels)
        bs = images.size(0)  # batch size
        total_loss += loss.item() * bs  # 누적
        total_acc  += accuracy_top1(logits, labels) * bs  # 누적
        n += bs  # 누적
    return total_loss / n, total_acc / n  # 평균 loss/acc

def plot_history(hist):
    epochs = np.arange(1, len(hist['train_loss'])+1)  # epoch index
    plt.figure(figsize=(12,4))
    plt.plot(epochs, hist['train_loss'], label='train_loss')  # train loss
    plt.plot(epochs, hist['val_loss'], label='val_loss')  # val loss
    plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(); plt.grid(True)
    plt.show()

    plt.figure(figsize=(12,4))
    plt.plot(epochs, hist['train_acc'], label='train_acc')  # train acc
    plt.plot(epochs, hist['val_acc'], label='val_acc')  # val acc
    plt.xlabel('epoch'); plt.ylabel('acc'); plt.legend(); plt.grid(True)
    plt.show()

In [ ]:
---
## 4) ResNet18 학습
- **목적:** CIFAR-10에서 ResNet18을 학습하고, 가장 좋은 성능의 모델을 저장합니다.
- **관찰 포인트**
    - `resnet18_cifar10.pth`가 있으면 **그 모델을 로드**하고 **10 epoch만 추가 학습**합니다.
    - 파일이 없으면 **20 epoch 학습**합니다.
    - 체크포인트로 실험을 이어갈 수 있다는 점

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=cfg.label_smoothing)
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.momentum, weight_decay=cfg.weight_decay)

ckpt_path = 'resnet18_cifar10.pth'  # best checkpoint 경로
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}  # 기록용 dict